In [1]:
import os
import time
import numpy as np
import tensorflow as tf


resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

strategy = tf.distribute.TPUStrategy(resolver)


def create_random_matrix(shape, dtype):
    if dtype == tf.float32:
        return np.random.rand(*shape).astype(np.float32)
    elif dtype == tf.int32:
        return np.random.randint(0, 100000, size=shape, dtype=np.int32)
    elif dtype == tf.float64:
        return np.random.rand(*shape).astype(np.float64)
    elif dtype == tf.int64:
        return np.random.randint(0, 100000, size=shape, dtype=np.int64)
    else:
        raise ValueError("Unsupported data type")


def multiply_matrices(a, b):
    result = tf.matmul(a,b)
    return result


data_types = [tf.int32, tf.float32, tf.int64, tf.float64]

output_file = "matrix_multiplication_results.txt"

with open(output_file, "w") as file:
    for dtype in data_types:
        
        matrix_a = create_random_matrix((4096, 4096), dtype)
        matrix_b = create_random_matrix((4096, 4096), dtype)

        
        with strategy.scope():
           
            distributed_matrix_a = tf.convert_to_tensor(matrix_a)
            distributed_matrix_b = tf.convert_to_tensor(matrix_b)

            
            start_time = time.time()

            
            result = multiply_matrices(distributed_matrix_a.numpy(), distributed_matrix_b.numpy())

            
            end_time = time.time()

        
        elapsed_time = end_time - start_time

        
        print(f"Time taken for {dtype}: {elapsed_time:.4f} seconds")

        
        file.write(f"Data type: {dtype}\n")
        file.write(f"Time taken: {elapsed_time:.4f} seconds\n")
        file.write("First 10x10 resultant matrix:\n")
        np.savetxt(file, result[:10, :10], fmt="%d" if dtype == tf.int32 or dtype == tf.int64 else "%.6f")
        file.write("\n\n")


Time taken for <dtype: 'int32'>: 1.6376 seconds
Time taken for <dtype: 'float32'>: 1.6508 seconds
Time taken for <dtype: 'int64'>: 3.4502 seconds
Time taken for <dtype: 'float64'>: 3.2885 seconds


In [7]:
import tensorflow as tf
import time


dim = 4096
dtype_list = [tf.int32, tf.int64, tf.float32, tf.float64]

for dtype in dtype_list:
    
    if dtype == tf.int32:
        matrix_a = tf.random.uniform((dim, dim), minval=0, maxval=1000, dtype=dtype)
        matrix_b = tf.random.uniform((dim, dim), minval=0, maxval=1000, dtype=dtype)
    else:
        matrix_a = tf.random.uniform((dim, dim), minval=0, maxval=1000000, dtype=dtype)
        matrix_b = tf.random.uniform((dim, dim), minval=0, maxval=1000000, dtype=dtype)

    
    start_time = time.time()

    
    result = tf.matmul(matrix_a, matrix_b)

    
    end_time = time.time()

    
    print("Data type:", dtype, "Time taken:", end_time - start_time, "seconds")

    
    print("First 10 numbers of the resultant matrix:")
    print(result[:10, :10].numpy())
    print("\n")


Data type: <dtype: 'int32'> Time taken: 9.60452127456665 seconds
First 10 numbers of the resultant matrix:
[[1011133445  988808166 1015982340 1020522941 1017846437 1025299373
  1017414083 1018071868 1000380773 1013041047]
 [ 999755655  990267056 1023519923 1013835446 1019273721 1020516368
  1007444236 1006119188  990239576 1008336389]
 [1009216006 1000670210 1024969468 1025348667 1022394384 1026766571
  1015134925 1026185713  999433357 1018449151]
 [ 983773045  978617381 1023271169 1012124376 1013549495 1010717744
  1004833756 1004193915  983532000 1010144310]
 [1025791600 1017449426 1044740831 1050348059 1035691949 1044817977
  1039799057 1051451078 1005186859 1031003068]
 [ 991555901  980446520 1019578897 1013420542 1010523525 1015253166
  1009568679 1005908007  992193043  996987658]
 [1023134057 1005623151 1031645387 1043750216 1028822156 1044952782
  1033363694 1028558156 1004185097 1034848102]
 [1015449394  993116539 1032879176 1038291035 1026815892 1030836219
  1016597574 1034762